In [153]:
#cleaned y dataset
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

Y = pd.read_csv("cvd_mortality_2018_2020.csv")
X = pd.read_csv("final.csv")

In [154]:
Y = Y.sort_values(by='statecounty').reset_index()
Y = Y.drop(columns='Unnamed: 0')
Y = Y.drop(columns = 'index')
Y

,statecounty,allrace_cvd_mortality_per_100k_2018_2020,api_cvd_mortality_per_100k_2018_2020
0,01Autauga County,401.0,NaN
1,01Baldwin County,353.7,205.4
2,01Barbour County,575.2,NaN
3,01Bibb County,503.7,107.2
4,01Blount County,463.6,104.7
...,...,...,...
3221,72Yabucoa Municipio,152.6,NaN
3222,72Yauco Municipio,275.8,NaN
3223,78Saint Croix (County Equivalent) County,NaN,NaN
3224,78Saint John (County Equivalent) County,NaN,NaN


In [155]:
X =  X.sort_values(by='statecounty').reset_index()
X = X.drop(columns = 'index')
X

,statecounty,Total_Violent_and_Property_Crimes_FBI2019,Total_Violent_and_Property_Crimes_Rate_FBI2019,Percent_Violent_Crimes_In_Total_Violent_and_Property_Crimes_FBI2019,Percent_Property_Crimes_In_Total_Violent_and_Property_Crimes_FBI2019,Total_Violent_Crimes_Rate_FBI2019,Total_Property_Crimes_Rate_FBI2019,"Black_AA_Heart_Attack_Death_Rate_Per_100,000_CDC_ehTracking_2018","Black_AA_Heart_Attack_Death_Rate_Per_100,000_CDC_ehTracking_2019","Black_AA_Heart_Attack_Death_Rate_Per_100,000_CDC_ehTracking_2020",...,Number_of_Farmers_Market_Accept_SFMNP_USDA_2018,Percent_of_Farmers_Market_Accept_SFMNP_USDA_2018,Number_of_Recreation_Fitness_Facilities_Per1000_Residents_USDA_2016,NHPI_Percent_Difference_From_Overall_Median_Household_Income_In_the_Past_Year_ACS2017_21,NHPI_Percent_Difference_From_Overall_Median_Family_Income_In_the_Past_Year_ACS2017_21,NHPI_Percent_Difference_From_Overall_Per_Capita_Income_In_the_Past_Year_ACS2017_21,NHPI_Percent_Difference_From_Overall_Median_Nonfamily_Household_Income_In_the_Past_Year_ACS2017_21,NHPI_Median_Earnings_Percent_Difference_From_Overall_Median_Earnings_In_the_Past_Year_ACS2017_21,NHPI_Male_Percent_Difference_From_Overall_Median_Earnings_In_the_Past_Year_ACS2017_21,NHPI_Female_Percent_Difference_From_Overall_Median_Earnings_In_the_Past_Year_ACS2017_21
0,01Autauga County,NaN,NaN,NaN,NaN,NaN,NaN,75.0,64.9,81.1,...,0.0,0.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01Baldwin County,NaN,NaN,NaN,NaN,NaN,NaN,54.8,48.0,60.0,...,4.0,100.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01Barbour County,NaN,NaN,NaN,NaN,NaN,NaN,62.4,55.8,61.3,...,2.0,50.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01Bibb County,NaN,NaN,NaN,NaN,NaN,NaN,40.9,34.7,36.3,...,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01Blount County,NaN,NaN,NaN,NaN,NaN,NaN,42.2,31.9,39.7,...,1.0,100.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3230,72Yauco Municipio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3231,74Midway Islands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3232,78St. Croix Island,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3233,78St. John Island,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
aligned = pd.concat([X['statecounty'], Y['statecounty']], axis = 1)
aligned.columns = ['x_county', 'y_county']
mismatch = aligned[aligned['x_county'] != aligned['y_county']]
mismatch

,x_county,y_county
67,02Aleutians East Borough,02Aleutians East
68,02Aleutians West Census Area,02Aleutians West
69,02Anchorage Municipality,02Anchorage
70,02Bethel Census Area,02Bethel
71,02Bristol Bay Borough,02Bristol Bay
...,...,...
3230,72Yauco Municipio,NaN
3231,74Midway Islands,NaN
3232,78St. Croix Island,NaN
3233,78St. John Island,NaN


In [157]:
X_3142 = X.iloc[:-93]
X_3142.replace([np.inf, -np.inf], np.nan, inplace = True)

/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_42813/392253245.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_3142.replace([np.inf, -np.inf], np.nan, inplace = True)


In [158]:
Y_3142 = Y.iloc[:-84]
Y_3142 = pd.DataFrame(Y_3142['allrace_cvd_mortality_per_100k_2018_2020'])
Y_3142.isna().sum()#3
imputer = KNNImputer(n_neighbors=10)
Y_impute = pd.DataFrame(imputer.fit_transform(Y_3142), columns = Y_3142.columns)
Y_impute.isna().sum()
#Y final version
scaler = StandardScaler()
Y_final = pd.DataFrame(scaler.fit_transform(Y_impute))
Y_final.columns = ['CVD death 2019_2020 std']
Y_final.to_csv("response.csv", index = False)

In [159]:
# kick out missing a lot columns
def filter_col(df, threshold = 0.3):
    missing_percentage = df.isna().mean()
    retain_keywords = ['transportation', 'physical', 'activity','income', 'poverty']
    retain_columns = df.columns[df.columns.str.contains('|'.join(retain_keywords), case=False)]
    filter_df = df.loc[:, (missing_percentage <= threshold) | (df.columns.isin(retain_columns))]
    return filter_df

X_filter = filter_col(X_3142, threshold= 0.2)
X_filter
X_filter = X_filter.select_dtypes(include = [np.number])
X_filter

,"NHPI_Heart_Attack_Death_Rate_Per_100,000_CDC_ehTracking_2018","NHPI_Heart_Attack_Death_Rate_Per_100,000_CDC_ehTracking_2019","NHPI_Heart_Attack_Death_Rate_Per_100,000_CDC_ehTracking_2020","NHPI_Heart_Attack_Death_Rate_Per_100,000_CDC_ehTracking_2021","NHPI_Ischemic_Heart_Disease_Death_Rate_Per_100,000_CDC_ehTracking_2018","NHPI_Ischemic_Heart_Disease_Death_Rate_Per_100,000_CDC_ehTracking_2019","NHPI_Ischemic_Heart_Disease_Death_Rate_Per_100,000_CDC_ehTracking_2021","Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2016_18","Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2018_20","Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2019_21",...,Percent_of_Farmers_Market_Accept_WIC_USDA_2018,Number_of_Farmers_Market_Accept_WIC_Cash_USDA_2018,Percent_of_Farmers_Market_Accept_SNAP_USDA_2018.1,Number_of_Farmers_Market_Accept_SFMNP_USDA_2018,Percent_of_Farmers_Market_Accept_SFMNP_USDA_2018,Number_of_Recreation_Fitness_Facilities_Per1000_Residents_USDA_2016,NHPI_Percent_Difference_From_Overall_Median_Household_Income_In_the_Past_Year_ACS2017_21,NHPI_Percent_Difference_From_Overall_Median_Family_Income_In_the_Past_Year_ACS2017_21,NHPI_Percent_Difference_From_Overall_Per_Capita_Income_In_the_Past_Year_ACS2017_21,NHPI_Percent_Difference_From_Overall_Median_Nonfamily_Household_Income_In_the_Past_Year_ACS2017_21
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,266.6,283.5,297.3,...,0.0,0.0,0.000000,0.0,0.0,6.0,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,NaN,0.0,NaN,242.5,237.0,239.6,...,0.0,0.0,0.000000,4.0,100.0,21.0,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,298.4,364.5,347.4,...,0.0,0.0,50.000000,2.0,50.0,0.0,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,315.3,325.8,332.4,...,0.0,0.0,0.000000,0.0,0.0,1.0,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,284.7,300.4,310.7,...,100.0,0.0,0.000000,1.0,100.0,4.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,177.7,168.0,150.1,...,0.0,0.0,0.000000,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,231.7,224.6,237.9,...,0.0,0.0,0.000000,0.0,0.0,6.0,NaN,NaN,NaN,NaN
3139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,109.5,116.4,117.2,...,0.0,0.0,33.333332,0.0,0.0,13.0,NaN,NaN,NaN,NaN
3140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,215.4,229.1,229.6,...,0.0,0.0,0.000000,0.0,0.0,2.0,NaN,NaN,NaN,NaN


In [160]:
X_filter.replace([np.inf, -np.inf], np.nan, inplace=True)

valid_columns = X_filter.loc[:, ~X_filter.isna().all()].columns
X_impute = X_filter[valid_columns]
X_impute 

,"NHPI_Heart_Attack_Death_Rate_Per_100,000_CDC_ehTracking_2018","NHPI_Heart_Attack_Death_Rate_Per_100,000_CDC_ehTracking_2019","NHPI_Heart_Attack_Death_Rate_Per_100,000_CDC_ehTracking_2020","NHPI_Heart_Attack_Death_Rate_Per_100,000_CDC_ehTracking_2021","NHPI_Ischemic_Heart_Disease_Death_Rate_Per_100,000_CDC_ehTracking_2018","NHPI_Ischemic_Heart_Disease_Death_Rate_Per_100,000_CDC_ehTracking_2019","NHPI_Ischemic_Heart_Disease_Death_Rate_Per_100,000_CDC_ehTracking_2021","Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2016_18","Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2018_20","Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2019_21",...,Percent_of_Farmers_Market_Accept_WIC_USDA_2018,Number_of_Farmers_Market_Accept_WIC_Cash_USDA_2018,Percent_of_Farmers_Market_Accept_SNAP_USDA_2018.1,Number_of_Farmers_Market_Accept_SFMNP_USDA_2018,Percent_of_Farmers_Market_Accept_SFMNP_USDA_2018,Number_of_Recreation_Fitness_Facilities_Per1000_Residents_USDA_2016,NHPI_Percent_Difference_From_Overall_Median_Household_Income_In_the_Past_Year_ACS2017_21,NHPI_Percent_Difference_From_Overall_Median_Family_Income_In_the_Past_Year_ACS2017_21,NHPI_Percent_Difference_From_Overall_Per_Capita_Income_In_the_Past_Year_ACS2017_21,NHPI_Percent_Difference_From_Overall_Median_Nonfamily_Household_Income_In_the_Past_Year_ACS2017_21
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,266.6,283.5,297.3,...,0.0,0.0,0.000000,0.0,0.0,6.0,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,NaN,0.0,NaN,242.5,237.0,239.6,...,0.0,0.0,0.000000,4.0,100.0,21.0,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,298.4,364.5,347.4,...,0.0,0.0,50.000000,2.0,50.0,0.0,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,315.3,325.8,332.4,...,0.0,0.0,0.000000,0.0,0.0,1.0,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,284.7,300.4,310.7,...,100.0,0.0,0.000000,1.0,100.0,4.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,177.7,168.0,150.1,...,0.0,0.0,0.000000,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,231.7,224.6,237.9,...,0.0,0.0,0.000000,0.0,0.0,6.0,NaN,NaN,NaN,NaN
3139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,109.5,116.4,117.2,...,0.0,0.0,33.333332,0.0,0.0,13.0,NaN,NaN,NaN,NaN
3140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,215.4,229.1,229.6,...,0.0,0.0,0.000000,0.0,0.0,2.0,NaN,NaN,NaN,NaN


In [161]:
exclude_word = ['cvd', 'death', 'mortality']
X_impute = X_impute.loc[:, ~X_impute.columns.str.contains('|'.join(exclude_word), case = False)]
X_impute

,Healthcare_General_Practice_Provider_Rate_per100000_HRSA2020,Healthcare_Family_Medicine_Provider_Rate_per100000_HRSA2020,Civilian_Noninstitutionalized_Population_For_Insurance_Coverage_ACS2016_20,Population_With_Health_Insurance_ACS2016_20,Population_Without_Health_Insurance_ACS2016_20,Percent_Population_Without_Health_Insurance_ACS2016_20,Population_18yrs_And_Under_Civilian_Noninstitutionalized_Population_For_Insurance_Coverage_ACS2016_20,Population_18yrs_And_Under_With_Health_Insurance_ACS2016_20,Population_18yrs_And_Under_Without_Health_Insurance_ACS2016_20,PercentPopulation_18yrs_And_Under_Without_Health_Insurance_ACS2016_20,...,Percent_of_Farmers_Market_Accept_WIC_USDA_2018,Number_of_Farmers_Market_Accept_WIC_Cash_USDA_2018,Percent_of_Farmers_Market_Accept_SNAP_USDA_2018.1,Number_of_Farmers_Market_Accept_SFMNP_USDA_2018,Percent_of_Farmers_Market_Accept_SFMNP_USDA_2018,Number_of_Recreation_Fitness_Facilities_Per1000_Residents_USDA_2016,NHPI_Percent_Difference_From_Overall_Median_Household_Income_In_the_Past_Year_ACS2017_21,NHPI_Percent_Difference_From_Overall_Median_Family_Income_In_the_Past_Year_ACS2017_21,NHPI_Percent_Difference_From_Overall_Per_Capita_Income_In_the_Past_Year_ACS2017_21,NHPI_Percent_Difference_From_Overall_Median_Nonfamily_Household_Income_In_the_Past_Year_ACS2017_21
0,1.8,26.7,54553.0,50208.0,4345.0,7.96,13804.0,13462.0,342.0,2.48,...,0.0,0.0,0.000000,0.0,0.0,6.0,NaN,NaN,NaN,NaN
1,2.6,34.0,215669.0,195168.0,20501.0,9.51,49241.0,46845.0,2396.0,4.87,...,0.0,0.0,0.000000,4.0,100.0,21.0,NaN,NaN,NaN,NaN
2,0.0,28.5,22116.0,19754.0,2362.0,10.68,5455.0,5293.0,162.0,2.97,...,0.0,0.0,50.000000,2.0,50.0,0.0,NaN,NaN,NaN,NaN
3,4.5,94.9,20748.0,18870.0,1878.0,9.05,4823.0,4751.0,72.0,1.49,...,0.0,0.0,0.000000,0.0,0.0,1.0,NaN,NaN,NaN,NaN
4,0.0,15.6,57274.0,51528.0,5746.0,10.03,13981.0,13392.0,589.0,4.21,...,100.0,0.0,0.000000,1.0,100.0,4.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,0.0,40.6,9846.0,8448.0,1398.0,14.20,2406.0,1982.0,424.0,17.62,...,0.0,0.0,0.000000,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3138,0.0,25.8,43076.0,37997.0,5079.0,11.79,11994.0,10825.0,1169.0,9.75,...,0.0,0.0,0.000000,0.0,0.0,6.0,NaN,NaN,NaN,NaN
3139,4.3,51.1,23296.0,20630.0,2666.0,11.44,4423.0,3988.0,435.0,9.83,...,0.0,0.0,33.333332,0.0,0.0,13.0,NaN,NaN,NaN,NaN
3140,0.0,19.8,20185.0,17912.0,2273.0,11.26,6010.0,5568.0,442.0,7.35,...,0.0,0.0,0.000000,0.0,0.0,2.0,NaN,NaN,NaN,NaN


In [162]:
X_impute.isna().sum().sum()#0
X_final = pd.DataFrame(scaler.fit_transform(X_impute),columns=X_impute.columns)
X_final.to_csv("predict.csv", index = False)
